## Descrição

Notebook exploratorio do solver e do método de validação de soluções

In [43]:
#import yaml
import copy
import numpy as np
import pandas as pd

import os
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from pyvrp import Model, Solution
from pyvrp.stop import MaxRuntime
from pyvrp import Client, Depot, ProblemData, VehicleType, solve as _solve
from pyvrp.plotting import plot_result

In [45]:
df=pd.read_csv("../data/niveis_inicio_fim_dez_1.csv", delimiter=',')

In [46]:
df.head()

,Data,Circuito,Tipo,local_id,pred_ench_3m_med,pred_ench_hgb,nivel_fim_hgb,nivel_fim_baseline
0,2024-12-02,Circ_Ilhas,Vidro,3863,2.159022,1.543212,1.543212,2.159022
1,2024-12-02,Circ_Ilhas,Vidro,3864,1.317753,1.775170,1.775170,1.317753
2,2024-12-02,Circ_Ilhas,Vidro,3844,1.135486,1.052217,1.052217,1.135486
3,2024-12-02,Circ_Ilhas,Vidro,3865,1.351893,1.089465,1.089465,1.351893
4,2024-12-02,Circ_Ilhas,Vidro,3732,1.158147,1.081761,1.081761,1.158147


In [47]:
def load_data(df):
    df['Data'] = pd.to_datetime(df['Data'])#, dayfirst=True)
    df['Data'] = df['Data'].dt.normalize()

Pré-processa o dataframe. Faz o seguinte: Converte a coluna 'Data' para datetime no formato dia/mes/ano e remove a parte da hora (dt.normalize).
Substitui os valores que são NaN nas colunas Nivel_inicio e Nivel_fim por 3. Depois para cada linha dessas colunas, pega nos valores. Se o valor da coluna Nivel_inicio for maior que o da Nivel_fim, iverte os valores.

In [48]:
CIRCUITOS = ['Circuito_01', 'Circuito_02', 'Circuito_03', 'Circuito_04',
             'Circuito_05', 'Circuito_06', 'Circuito_07', 'Circuito_08',
             'Circuito_09', 'Circuito_10', 'Circuito_11', 'Circuito_12', 'Circ_Ilhas']

In [49]:
tpath = '../sim/environments/gcvrp/sim_data/matrizes_tempo/'
dpath = '../sim/environments/gcvrp/sim_data/matrizes_distancia/'
cpath = '../sim/environments/gcvrp/sim_data/circuitos/'

mat_temp = {circ: np.loadtxt(tpath + f'matriz_tempo_{circ}.csv', delimiter=',')  for circ in CIRCUITOS} # minutes
mat_dist = {circ: np.loadtxt(dpath + f'matriz_dist_{circ}.csv', delimiter=',')  for circ in CIRCUITOS} # mete
circuitos = {circ: pd.read_csv(cpath + f'{circ}_pontos.csv')  for circ in CIRCUITOS}        

In [50]:
def construir_instance(circuito, pontos, matrizes_dist_dict, matrizes_tempo_dict, demands, capacidade):
    coords = [[p['lon'], p['lat']] for k, p in pontos.iterrows()]
    num_locs = len(coords)
    instance = {
        "coords": coords,
        "demands": demands,
        "prize": [beta * d / sum(demands) for d in demands],  # ← CORRIGIDO AQUI
        "service_time": [5] * num_locs,
        "capacity": capacidade,
        "cost_matrix": matrizes_dist_dict[circuito] / dist_scale ,
        "time_matrix": matrizes_tempo_dict[circuito],
        "start_depot_idx": 0,
        "end_depot_idx": 1
    }

    return instance

Constroi a instance com os dados 

In [51]:
def scale(data, scaling_factor):
    arr = np.array(data, dtype=float)  # converte input para array numpy float
    arr = arr * scaling_factor          # multiplica valores, não o tamanho
    arr = np.where(np.isfinite(arr), arr, 0)  # trata NaN ou inf substituindo por 0
    #arr = arr.round().astype(int)       # arredonda e converte para int
    if arr.size == 1:
        return arr.item()               # retorna escalar se só tiver um elemento
    return arr

In [52]:
def instance2data(instance: dict, scaling_factor: int, verbose=False) -> ProblemData:
    """
    Converte uma instância genérica para o formato ProblemData do PyVRP,
    preparada para problemas com clientes opcionais (PCVRP).
    """


    # Converter DataFrames para listas, se necessário
    #if isinstance(instance["cost_matrix"], pd.DataFrame):
    #    instance["cost_matrix"] = instance["cost_matrix"].values.tolist()
    #if isinstance(instance["time_matrix"], pd.DataFrame):
    #    instance["time_matrix"] = instance["time_matrix"].values.tolist()

    # Aplicar escala
    coords = scale(instance["coords"], scaling_factor)
    demands = scale(instance["demands"], scaling_factor)
    prize = scale(instance["prize"], scaling_factor)
    service = scale(instance["service_time"], scaling_factor)
    capacity = [scale(instance["capacity"], scaling_factor)]
    matrix = scale(instance["cost_matrix"], scaling_factor)
    matrix_time = scale(instance["time_matrix"], scaling_factor)

    num_locs = len(coords)
    start_depot_idx = instance.get("start_depot_idx", 0)
    end_depot_idx = instance.get("end_depot_idx", 1)

    # Verificar índices de depósitos
    if start_depot_idx < 0 or start_depot_idx >= num_locs:
        raise ValueError(f"Início do depósito inválido: {start_depot_idx}")
    if end_depot_idx < 0 or end_depot_idx >= num_locs:
        raise ValueError(f"Fim do depósito inválido: {end_depot_idx}")

    # Criar depósitos
    depots = [Depot(x=coords[start_depot_idx][0], y=coords[start_depot_idx][1])]
    if start_depot_idx != end_depot_idx:
        depots.append(Depot(x=coords[end_depot_idx][0], y=coords[end_depot_idx][1]))

    # Criar clientes opcionais (com prize)
    clients = []
    for idx in range(num_locs):
        if idx not in [start_depot_idx, end_depot_idx]:
            clients.append(Client(
                x=coords[idx][0],
                y=coords[idx][1],
                pickup=[demands[idx]],
                prize=prize[idx],
                service_duration=service[idx],
                required=False,  # ← ESSENCIAL para PCVRP
            ))

    # Tipo de veículo
    vehicle_types = [
        VehicleType(
            num_available=1,
            max_duration=420*PYVRP_SCALING_FACTOR,
            capacity=capacity,
            start_depot=start_depot_idx,
            end_depot=end_depot_idx,
        )
    ]

    # Verificação da matriz
    expected_size = num_locs
    if matrix.shape != (expected_size, expected_size):
        raise ValueError(f"Shape inconsistente: matriz {matrix.shape}, esperava {(expected_size, expected_size)}")

    if verbose:
        # Debug info
        print("→ Matrix shape:", matrix.shape)
        print("→ Num clients:", len(clients))
        print("→ Num depots:", len(depots))
        print("→ Total demand:", sum(demands))
        print("Prize: ", prize)
        print("Demands: ", demands)
        print("→ Expected matrix size:", expected_size)
        print("OPAAAA", matrix)
        print("OLAAA", matrix_time)

    return ProblemData(clients, depots, vehicle_types, [matrix], [matrix_time])

In [53]:
def solution2action(solution: Solution) -> list[int]:
    """
    Converte uma solução do PyVRP para a representação de ações (tour gigante),
    incluindo explicitamente os índices de depósito inicial e final.
    """
    action = []
    
    for route in solution.routes():
        # Adicionar o depósito inicial
        action.append(route.start_depot())
        
        # Adicionar as visitas dos clientes
        action.extend(route.visits())
        
        # Adicionar o depósito final
        action.append(route.end_depot())
        
    return action


In [54]:
def solve(instance, max_runtime, **kwargs):
    data = instance2data(instance, PYVRP_SCALING_FACTOR)
    stop = MaxRuntime(max_runtime)
    result = _solve(data, stop, **kwargs) # A função de resolução retorna um Result
    solution = result.best # A melhor solução está no atributo 'best'
    cost = result.cost() / PYVRP_SCALING_FACTOR # Assumindo que 'cost' é um atributo da Solution
    #print("→ Prize total recolhido:", solution.prize())
    #print("→ Custo (distância total):", solution.cost())
    
    return solution, cost

In [55]:
def analisar_solucao(solution, cost, demands, beta, dist_scale, scaling_factor):
    print("Função Objetivo:", cost)

    # Ações (rota)
    actions = solution2action(solution)
    print("Ações da solução:", actions)

    # Distância total ajustada
    distancia_ajustada = solution.distance() / scaling_factor


    # Prêmios não coletados ajustados
    premios_nao_coletados = solution.uncollected_prizes() / scaling_factor
    #print("Prêmios não coletados ajustados:", premios_nao_coletados)

    # Custo total considerando distância e prêmios não coletados
    custo_total = distancia_ajustada + (solution.uncollected_prizes() / scaling_factor)
    #print("Custo total ajustado (distância + prêmios não coletados):", custo_total)

    # Distância ajustada com fator de escala
    distancia_escalada = distancia_ajustada * dist_scale
    print("Distância:", distancia_escalada)

    # Duração ajustada
    duracao_ajustada = solution.duration() / scaling_factor
    print("Duração:", duracao_ajustada)

    premios_un = solution.uncollected_prizes() * sum(demands) / (beta * scaling_factor)
    print("Prêmios não coletados:", premios_un)

    
    # Prêmios não coletados em % do beta
    percentual_nao_coletado = solution.uncollected_prizes() / (beta * scaling_factor) * 100
    print("Prêmios não coletados (%):", percentual_nao_coletado)

    # Prêmios coletados ajustados
    premios_coletados = solution.prizes() * sum(demands) / (beta * scaling_factor)
    print("Prêmios coletados:", premios_coletados)

    soma_total_lixo= sum(demands)
    print("Total de lixo:", soma_total_lixo)

    return {
        "distancia_ajustada": distancia_ajustada,
        "penalidade_premios": premios_un,
        "distancia_escalada": distancia_escalada,
        "duracao_ajustada": duracao_ajustada,
        "percentual_nao_coletado": percentual_nao_coletado,
        "premios_coletados": premios_coletados,
        "custo_total": custo_total,
        "acoes": actions,
    }


In [56]:
def calc_sub_rota(actions, demands_fim, matrizes_dist_dict, matrizes_tempo_dict , circuito, verbose=False):

    circuito_esc_tempo=matrizes_tempo_dict[circuito]
    circuito_esc_dist=matrizes_dist_dict[circuito]

    service_time = 5
    sub_rota = [actions[0]]  # incluir o depósito inicial
    tempo = 0
    dist = 0
    curr_load = 0
    cap = 120

    for k in range(1, len(actions)-1):
        serv_ant = actions[k - 1]
        serv = actions[k]
        #print(serv)
        demanda = demands_fim[serv]
        if verbose:
            print("O ponto é", serv)
            print("A sua demanda é", demanda)

            print(f"curr_load: {curr_load} ({type(curr_load)})")
            print(f"demanda: {demanda} ({type(demanda)})")
            print(f"capacidade: {cap} ({type(cap)})")
            print(f"soma: {curr_load + demanda}")
            print(f"Condição: {curr_load + demanda <= cap}")
        if curr_load + demanda <= cap:
            curr_load += demanda
            sub_rota.append(serv)
            tempo += circuito_esc_tempo[serv_ant, serv]
            tempo += service_time
            dist += circuito_esc_dist[serv_ant, serv]

        else:
            carga_possivel = cap - curr_load

            if carga_possivel > 0:
                #print("A carga é",carga_possivel)
                curr_load += carga_possivel
                tempo += circuito_esc_tempo[serv_ant, serv]
                tempo += service_time
                dist += circuito_esc_dist[serv_ant, serv]
                break
    # Finaliza a rota
    sub_rota.append(actions[-1])
    tempo += circuito_esc_tempo[serv, actions[-1]]
    dist += circuito_esc_dist[serv, actions[-1]]
    return sub_rota, tempo, dist, curr_load

In [57]:
PYVRP_SCALING_FACTOR = 1_000_000
beta = 10
dist_scale = 20000

In [58]:
load_data(df)

In [25]:
#circuitos_matrizes_dist

In [59]:
data=pd.to_datetime('2024-12-02')
circuito='Circ_Ilhas'
tipo='Vidro'
capacidade=120
pontos_do_circuito = circuitos[circuito]


In [60]:
#demands_inicio = extrair_demands(df, data= data.date(), circuito=circuito, tipo=tipo, pontos_do_circuito=pontos_do_circuito, coluna='pred_ench_3m_med')

In [61]:
df_filtrado = df[
        (df['Data'].dt.date == data) &
        (df['Circuito'] == circuito) &
        (df['Tipo'] == tipo)
    ]

C:\Users\simao\AppData\Local\Temp\ipykernel_22380\122197432.py:2: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  (df['Data'].dt.date == data) &


In [62]:
demands_inicio_base = np.hstack([np.zeros(2), df_filtrado['pred_ench_3m_med'].to_numpy()]) 
demands_inicio_hgb = np.hstack([np.zeros(2), df_filtrado['pred_ench_hgb'].to_numpy()]) 

In [63]:
demands_inicio_hgb.sum()

70.98883448991012

In [64]:
instance_inicio_base = construir_instance(
    circuito = circuito,
    pontos = pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands = demands_inicio_base,
    capacidade = capacidade
)

In [65]:
instance_inicio_hgb = construir_instance(
    circuito = circuito,
    pontos = pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands = demands_inicio_hgb,
    capacidade = capacidade
)

In [66]:
solution_inicio_base, cost_inicio_base= solve(instance_inicio_base, max_runtime=10)
print("Custo base (distância):", cost_inicio_base)
solution_inicio_hgb, cost_inicio_hgb= solve(instance_inicio_hgb, max_runtime=10)
print("Custo hgb (distância):", cost_inicio_hgb)

Custo base (distância): 2.586973
Custo hgb (distância): 2.592889


In [67]:
actions_inicio_base = solution2action(solution_inicio_base)
actions_inicio_hgb = solution2action(solution_inicio_hgb)

### sub rota + rota final 

In [68]:
demands_fim = np.hstack([np.zeros(2), df_filtrado['nivel_fim_hgb'].to_numpy()]) 

In [69]:
sum(demands_fim)

71.73652290068863

In [70]:
instance_fim = construir_instance(
    circuito=circuito,
    pontos=pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands=demands_fim,
    capacidade=capacidade
)

In [71]:
sub_rota_base, tempo_base, dist_base, curr_load_base = calc_sub_rota(actions_inicio_base, demands_fim, mat_dist,mat_temp, circuito)
sub_rota_hgb, tempo_hgb, dist_hgb, curr_load_hgb = calc_sub_rota(actions_inicio_hgb, demands_fim, mat_dist, mat_temp, circuito)

In [72]:
solution_fim, cost_fim = solve(instance_fim, max_runtime=10)
print("Custo (distância):", cost_fim)

Custo (distância): 2.592889


In [73]:
print("Rota Inicio base")
print("")

resultados_inicio = analisar_solucao(
    solution_inicio_base,
    cost_inicio_base,
    demands_inicio_base,
    beta,
    dist_scale,
    PYVRP_SCALING_FACTOR
)
print("")
print("--------------------------")
print("")


print("Rota Inicio hgb")
print("")

resultados_inicio = analisar_solucao(
    solution_inicio_hgb,
    cost_inicio_hgb,
    demands_inicio_hgb,
    beta,
    dist_scale,
    PYVRP_SCALING_FACTOR
)
print("")
print("--------------------------")
print("")
print("Subrota base")
print("")

print("Ações", sub_rota_base)
print("Distancia", dist_base)
print("Duração", tempo_base)
prizes_un_base=sum(demands_fim)-curr_load_base
print("Prêmios não coletados", prizes_un_base)
racio_base=prizes_un_base/sum(demands_fim) *100
print("Prêmios não coletados (%)", racio_base)
print("Prêmios coletados", curr_load_base)
print("Total de lixo", sum(demands_fim))

print("")
print("--------------------------")
print("")


print("Subrota hgb")
print("")

print("Ações", sub_rota_hgb)
print("Distancia", dist_hgb)
print("Duração", tempo_hgb)
prizes_un_hgb=sum(demands_fim)-curr_load_hgb
print("Prêmios não coletados", prizes_un_hgb)
racio_hgb=prizes_un_hgb/sum(demands_fim) *100
print("Prêmios não coletados (%)", racio_hgb)
print("Prêmios coletados", curr_load_hgb)
print("Total de lixo", sum(demands_fim))

print("")
print("--------------------------")
print("")

print("Rota Fim")
print("")
resultados_fim = analisar_solucao(
    solution_fim,
    cost_fim,
    demands_fim,
    beta,
    dist_scale,
    PYVRP_SCALING_FACTOR
)



Rota Inicio base

Função Objetivo: 2.586973
Ações da solução: [0, 28, 25, 14, 13, 15, 40, 11, 16, 43, 41, 42, 12, 19, 20, 18, 26, 27, 22, 23, 21, 29, 24, 36, 30, 39, 34, 33, 35, 2, 3, 9, 8, 4, 5, 10, 7, 6, 32, 38, 37, 31, 1]
Distância: 49631.36000000001
Duração: 310.777148
Prêmios não coletados: 0.9999960507134188
Prêmios não coletados (%): 1.05405
Prêmios coletados: 93.87160878026447
Total de lixo: 94.87178508736956

--------------------------

Rota Inicio hgb

Função Objetivo: 2.592889
Ações da solução: [0, 28, 25, 14, 13, 40, 15, 11, 16, 17, 43, 41, 42, 12, 19, 20, 18, 26, 27, 22, 23, 29, 21, 24, 36, 30, 39, 34, 33, 35, 2, 3, 9, 8, 4, 5, 10, 6, 7, 32, 38, 37, 31, 1]
Distância: 51857.78
Duração: 321.63148
Prêmios não coletados: 0.0
Prêmios não coletados (%): 0.0
Prêmios coletados: 70.9886570178239
Total de lixo: 70.98883448991012

--------------------------

Subrota base

Ações [0, 28, 25, 14, 13, 15, 40, 11, 16, 43, 41, 42, 12, 19, 20, 18, 26, 27, 22, 23, 21, 29, 24, 36, 30, 39, 34,